<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [153]:
import pandas as pd
import numpy

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import category_encoders as ce


from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


# fix random seed for reproducibility
numpy.random.seed(42)

In [105]:
# load data and create df
path = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)

# numerical features
numerical_features = df.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = df.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 150].index.tolist()

# Combine Categorical and numerical features
features = numerical_features + categorical_features 

# encode categoricals
churn_dict = {'Yes':1, 'No':0}
df['Churn'] = df['Churn'].map(churn_dict)
encoder = ce.OneHotEncoder()
df = encoder.fit_transform(df[features])

features = df.drop(columns='Churn').columns.tolist()
target = 'Churn'

# # train / test split
train, test = train_test_split(df, test_size = 0.2, random_state=22)

# # X, y matrix
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5634, 44) (5634,) (1409, 44) (1409,)


In [107]:
print(X_train)

[[-0.4357324  -0.29024187 -1.49304973 ... -0.54548944 -0.52710604
   1.92836536]
 [-0.4357324   0.80586073 -1.51141491 ...  1.83321606 -0.52710604
  -0.51857393]
 [ 2.29498656 -0.73680219  0.34513864 ... -0.54548944 -0.52710604
  -0.51857393]
 ...
 [ 2.29498656 -0.16845269  1.38694568 ... -0.54548944 -0.52710604
  -0.51857393]
 [-0.4357324  -0.69620579 -1.47802366 ... -0.54548944 -0.52710604
   1.92836536]
 [-0.4357324   1.45540301 -1.3561456  ... -0.54548944  1.89715146
  -0.51857393]]


In [136]:
inputs = X_train.shape[1]

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
model.fit(X_train, y_train, 
          validation_split=0.33, 
          epochs=10, 
          batch_size=10)

# Test Accuracy
scores = model.evaluate(X_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]}')

Train on 3774 samples, validate on 1860 samples
Epoch 1/10
3774/3774 [==============================] - 3s 772us/sample - loss: 1.0477 - acc: 0.7536 - val_loss: 1.0584 - val_acc: 0.7640
Epoch 2/10
3774/3774 [==============================] - 2s 400us/sample - loss: 0.8067 - acc: 0.7787 - val_loss: 0.7323 - val_acc: 0.7909
Epoch 3/10
3774/3774 [==============================] - 2s 444us/sample - loss: 0.6395 - acc: 0.7944 - val_loss: 0.7581 - val_acc: 0.7774
Epoch 4/10
3774/3774 [==============================] - 2s 593us/sample - loss: 0.6395 - acc: 0.7787 - val_loss: 0.8186 - val_acc: 0.7360
Epoch 5/10
3774/3774 [==============================] - 2s 485us/sample - loss: 0.5929 - acc: 0.7970 - val_loss: 0.7047 - val_acc: 0.7763
Epoch 6/10
3774/3774 [==============================] - 2s 448us/sample - loss: 0.7163 - acc: 0.7907 - val_loss: 0.6992 - val_acc: 0.7667
Epoch 7/10
3774/3774 [==============================] - 2s 438us/sample - loss: 0.9940 - acc: 0.7613 - val_loss: 0.9876 - va

## Make New Predictions and Test Accuracy

In [137]:
# Make predictions 
y_pred = model.predict(X_test)
y_pred = [1 if i>0.5 else 0 for i in y_pred]

# Test Accuracy
score = accuracy_score(y_test, y_pred)
print('accuracy score:', score)

accuracy score: 0.7686302342086586


## Tune Hyperparameters using GridSearchCV

#### Batch Size

In [138]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(inputs,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='relu'))  
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7887823979059855 using {'batch_size': 60, 'epochs': 20}
Means: 0.7882499098777771, Stdev: 0.0017571021144191722 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7676606376965841, Stdev: 0.02384633344547162 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7831025918324789, Stdev: 0.014617108227276744 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7887823979059855, Stdev: 0.012826287850372778 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7877174218495687, Stdev: 0.007889993006359076 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7886048952738444, Stdev: 0.010279314433816437 with: {'batch_size': 100, 'epochs': 20}


In [139]:
# Make predictions 
y_pred = grid.predict(X_test)
y_pred = [1 if i>0.5 else 0 for i in y_pred]

# Test Accuracy
score = accuracy_score(y_test, y_pred)
print('accuracy score:', score)

accuracy score: 0.7672107877927609


#### Optimizer (best = 'adam')

In [149]:
lr_list = [.001, .01, .1, .2, .3, .5]
for i in lr_list:
    def create_model():
        # create model
        model = Sequential()
        model.add(Dense(64, input_shape=(inputs,), activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='relu'))  
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(lr=i), metrics=['accuracy'])
        return model

    # create model
    model = KerasClassifier(build_fn=create_model, verbose=0)

    # define the grid search parameters
    param_grid = {'batch_size': [60],
                  'epochs': [20]}

    # Create Grid Search
    grid = GridSearchCV(estimator=model, cv=3, param_grid=param_grid, n_jobs=1)
    grid_result = grid.fit(X_train, y_train)

    # Report Results
    print('\nlearning rate: {}'.format(i))
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f"Means: {mean}, Stdev: {stdev} with: {param}") 


learning rate: 0.001
Best: 0.7754703760147095 using {'batch_size': 60, 'epochs': 20}
Means: 0.7754703760147095, Stdev: 0.013146538942099429 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.01
Best: 0.7447639505068461 using {'batch_size': 60, 'epochs': 20}
Means: 0.7447639505068461, Stdev: 0.017554842781877994 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.1
Best: 0.738551656405131 using {'batch_size': 60, 'epochs': 20}
Means: 0.738551656405131, Stdev: 0.008052025851505478 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.2
Best: 0.5722399751345316 using {'batch_size': 60, 'epochs': 20}
Means: 0.5722399751345316, Stdev: 0.2276735378305616 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.3
Best: 0.5821796158949534 using {'batch_size': 60, 'epochs': 20}
Means: 0.5821796158949534, Stdev: 0.2242774172794499 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.5
Best: 0.5821796158949534 using {'batch_size': 60, 'epochs': 20}
Means: 0.5821796158949534, St

In [150]:
lr_list = [.001, .01, .1]
for i in lr_list:
    def create_model():
        # create model
        model = Sequential()
        model.add(Dense(64, input_shape=(inputs,), activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='relu'))  
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Nadam(lr=i), metrics=['accuracy'])
        return model

    # create model
    model = KerasClassifier(build_fn=create_model, verbose=0)

    # define the grid search parameters
    param_grid = {'batch_size': [60],
                  'epochs': [20]}

    # Create Grid Search
    grid = GridSearchCV(estimator=model, cv=3, param_grid=param_grid, n_jobs=1)
    grid_result = grid.fit(X_train, y_train)

    # Report Results
    print('\nlearning rate: {}'.format(i))
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f"Means: {mean}, Stdev: {stdev} with: {param}") 


learning rate: 0.001
Best: 0.7877174417177836 using {'batch_size': 60, 'epochs': 20}
Means: 0.7877174417177836, Stdev: 0.0029594042149341682 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.01
Best: 0.7428115010261536 using {'batch_size': 60, 'epochs': 20}
Means: 0.7428115010261536, Stdev: 0.006133160987832967 with: {'batch_size': 60, 'epochs': 20}

learning rate: 0.1
Best: 0.7387291391690572 using {'batch_size': 60, 'epochs': 20}
Means: 0.7387291391690572, Stdev: 0.007889990324635276 with: {'batch_size': 60, 'epochs': 20}


In [151]:
# Make predictions 
y_pred = grid.predict(X_test)
y_pred = [1 if i>0.5 else 0 for i in y_pred]

# Test Accuracy
score = accuracy_score(y_test, y_pred)
print('accuracy score:', score)

accuracy score: 0.7182398864442867


### Dropout

In [155]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(inputs,), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='relu'))  
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# Dropout=0.2 makes the score worse
# Dropout=0.1 makes the score worse

Best: 0.7694355845451355 using {'batch_size': 60, 'epochs': 20}
Means: 0.7694355845451355, Stdev: 0.009789552249731895 with: {'batch_size': 60, 'epochs': 20}


### Activation Functions

In [157]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(inputs,), activation='sigmoid'))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))  
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, cv=5, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

# sigmoid is better than relu

Best: 0.8107916238272406 using {'batch_size': 60, 'epochs': 20}
Means: 0.8107916238272406, Stdev: 0.011391109243850112 with: {'batch_size': 60, 'epochs': 20}


### Epochs

In [ ]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(inputs,), activation='sigmoid'))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))  
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model)

# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [100]}

# Create Grid Search
grid = GridSearchCV(estimator=model, cv=5, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/100
4507/4507 [==============================] - 2s 342us/sample - loss: 0.5431 - acc: 0.7189
Epoch 2/100
4507/4507 [==============================] - 0s 94us/sample - loss: 0.4477 - acc: 0.7790
Epoch 3/100
4507/4507 [==============================] - 0s 94us/sample - loss: 0.4311 - acc: 0.7968
Epoch 4/100
4507/4507 [==============================] - 0s 94us/sample - loss: 0.4258 - acc: 0.7972
Epoch 5/100
4507/4507 [==============================] - 0s 93us/sample - loss: 0.4222 - acc: 0.7996
Epoch 6/100
4507/4507 [==============================] - 0s 94us/sample - loss: 0.4195 - acc: 0.8052
Epoch 7/100
4507/4507 [==============================] - 0s 95us/sample - loss: 0.4194 - acc: 0.8034
Epoch 8/100
4507/4507 [==============================] - 0s 95us/sample - loss: 0.4172 - acc: 0.8010
Epoch 9/100
4507/4507 [==============================] - 0s 96us/sample - loss: 0.4173 - acc: 0.8063
Epoch 10/100
4507/4507 [==============================] - 0s 97us/sample - loss: 0.4156 - 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?